In [85]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
from keras import layers, models
from keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop, SGD
from keras.layers import Dense,Conv2D, Dropout,Activation,MaxPooling2D,Flatten


In [86]:
atributos = '/Users/luceromartinezbonilla/Downloads/rec_fac/list_attr_celeba.txt'
atributos_finales = '/Users/luceromartinezbonilla/Downloads/rec_fac/list_attr_celeba_modificado.txt' 
with open(atributos, 'r') as f:
    print("skipping: " + f.readline())
    print("skipping headers: " + f.readline())
    with open(atributos_finales, 'w') as newf:
        for line in f:
            new_line = ' '.join(line.split())
            newf.write(new_line)
            newf.write('\n')
        

skipping: 202599

skipping headers: 5_o_Clock_Shadow Arched_Eyebrows Attractive Bags_Under_Eyes Bald Bangs Big_Lips Big_Nose Black_Hair Blond_Hair Blurry Brown_Hair Bushy_Eyebrows Chubby Double_Chin Eyeglasses Goatee Gray_Hair Heavy_Makeup High_Cheekbones Male Mouth_Slightly_Open Mustache Narrow_Eyes No_Beard Oval_Face Pale_Skin Pointy_Nose Receding_Hairline Rosy_Cheeks Sideburns Smiling Straight_Hair Wavy_Hair Wearing_Earrings Wearing_Hat Wearing_Lipstick Wearing_Necklace Wearing_Necktie Young 



In [75]:
tf.version.VERSION

'2.9.1'

In [87]:
#hacemos un csv
atributos = '/Users/luceromartinezbonilla/Downloads/rec_fac/list_attr_celeba_modificado.txt'
df = pd.read_csv(atributos, sep=' ',  header=None)

In [77]:
#visualizamos el dataframe de atributos
df.head()

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1


In [97]:
df.replace(to_replace=-1, value=0, inplace=True) #reemplazamos los valores "-1" por 0
#puede ser haya errores de formato, por lo que nos aseguramos reemplazando todo a numeros
df.replace(to_replace='0', value=0, inplace=True) 
df.replace(to_replace='-1', value=0, inplace=True) 
df.replace(to_replace='1', value=1, inplace=True) 

#Se toman todas las filas, y todas las columnas menos la primera y se transforma a una matriz numpy
x = np.asarray(df.iloc[:, 1:]).astype('int64')
print(x)


[[0 1 1 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 1]
 [0 1 1 ... 0 0 1]
 [0 1 1 ... 0 0 1]]


In [ ]:
files = tf.data.Dataset.from_tensor_slices(df[0]) 
attributes = tf.data.Dataset.from_tensor_slices(x) 
data = tf.data.Dataset.zip((files, attributes)) #Se concatena la columna de nombres (files) y la matriz de atributos (x)


In [ ]:
path_to_images = '/Users/luceromartinezbonilla/Desktop/redes neuronales/reconocimiento_facial/archive/img_align_celeba/'

def process_file(file_name, attributes):
    #Se cargan las imágenes uniendo la ruta de la carpeta y el nombre de la imagen
    image = tf.io.read_file(path_to_images + file_name)
    #Se descomprime la imagen
    image = tf.image.decode_jpeg(image, channels=3)
    #Se reescala la imagen
    image = tf.image.resize(image, [192, 192])
    #Se normalizan los pixeles
    image /= 255.0
    #Esta función regresa a la imagen en lugar del nombre obtenido de file_name
    #y los atributos de la imagen sin modificar
    return image, attributes

In [99]:
#Se aplica la función process_file al Dataset unido 
labeled_images = data.map(process_file).batch(batch_size)

In [96]:
labeled_images

<BatchDataset element_spec=(TensorSpec(shape=(None, 192, 192, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 40), dtype=tf.int64, name=None))>

In [90]:
epochs = 3
batch_size = 30
optimizer = 'rmsprop'
ih, iw = 192, 192

In [91]:
#usamos una separacion de los datos 80-20
num_train = int(len(df)*0.8) #80% de los datos 
num_test =len(df) - num_train #20% de los datos

epochs_steps = num_train // batch_size
test_steps = num_test // batch_size

d_train = imagen_etiquetada.take(num_train)
d_test = imagen_etiquetada.skip(num_train)


In [92]:
model = Sequential()

#capas convolucionales
model.add(Conv2D(40, (3, 3), input_shape= (192,192,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(100, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(150, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#aplanamiento 
model.add(Flatten())

#redes densas
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.2))

#Capa de salida con 40 neuronas
model.add(Dense(40))
model.add(Activation('sigmoid'))

In [93]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['binary_accuracy'])

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 190, 190, 40)      1120      
                                                                 
 activation_25 (Activation)  (None, 190, 190, 40)      0         
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 95, 95, 40)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 93, 93, 100)       36100     
                                                                 
 activation_26 (Activation)  (None, 93, 93, 100)       0         
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 46, 46, 100)      0         
 g2D)                                                 

In [94]:
#entrenamos el mode
model.fit(d_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=d_test,
    validation_steps=test_steps)


model.save('red_atributos.h5')

Epoch 1/3


AttributeError: in user code:

    File "/Users/luceromartinezbonilla/opt/anaconda3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/Users/luceromartinezbonilla/opt/anaconda3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/luceromartinezbonilla/opt/anaconda3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/Users/luceromartinezbonilla/opt/anaconda3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/luceromartinezbonilla/opt/anaconda3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 948, in c